In [ ]:
import sys
import os

import pandas as pd
import geopandas as gpd

sys.path.append(r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.shapeMisc as shapeMisc
import GOSTrocks.dataMisc as dMisc
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

In [ ]:
# Define source data
FUA_file = r"C:\WBG\Work\data\URBAN\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"
admin_lines = r"C:\WBG\Work\AdminBounds\Admin0_Polylines_CntryandDisp.shp"

out_folder = r"C:\WBG\Work\MENA_Urban\RESULTS\SHAPE"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
inFUA = gpd.read_file(FUA_file)
inAdmin = gpd.read_file(admin_lines)
inAdmin = inAdmin.to_crs(inFUA.crs)
admin_shape = inAdmin.union_all()

In [ ]:
# calculate shape metrics
inFUA["area"] = inFUA["geometry"].area
inFUA["perimeter"] = inFUA["geometry"].length
tPrint("Starting Polsby Popper")
inFUA["polsby"] = inFUA["geometry"].apply(lambda x: shapeMisc.polsby_popper(x))
tPrint("Starting Schwartzberg")
inFUA["schwartzberg"] = inFUA["geometry"].apply(lambda x: shapeMisc.schwartzberg(x))
tPrint("Finished")
inFUA.head()

In [ ]:
sel_results = inFUA.loc[:, ["eFUA_ID", "area", "perimeter", "polsby", "schwartzberg"]]
pd.DataFrame(sel_results).to_csv(os.path.join(out_folder, "FUA_shape_metrics.csv"))

In [ ]:
# Calculate distance from FUAs to nearest admin boundary
base_url = "https://geowb.worldbank.org/hosting/rest/services/Hosted/WB_GAD_Medium_Resolution/FeatureServer"
layer = 0
admin_lines = dMisc.gdf_esri_service(base_url, layer)

tPrint("Starting distance to admin")
admin_lines = admin_lines.to_crs(inFUA.crs)
fua_joined = gpd.sjoin_nearest(inFUA, admin_lines, distance_col="dist_to_admin")
fua_summarized = fua_joined.groupby("eFUA_ID").agg(
    {"dist_to_admin": "min", "eFUA_name": "first", "Cntry_ISO": "first"}
)
fua_summarized.sort_values("dist_to_admin", ascending=True).head(10)

In [ ]:
fua_summarized.to_csv(os.path.join(out_folder, "FUA_dist_to_admin.csv"))